In [0]:
%sql
-- Movie dimension
CREATE TABLE netflix_data.data_warehouse.DimMovie(
    MovieID INT PRIMARY KEY,
    MovieTitle VARCHAR(255),
    AverageRatingOfMovie DECIMAL(3,2)
);

In [0]:
%sql
INSERT INTO netflix_data.data_warehouse.dimmovie(MovieID, MovieTitle,AverageRatingOfMovie)
SELECT DISTINCT
    MovieID,
    MovieTitle,
    AverageRatingOfMovie
FROM netflix_data.transformed_data.bigflattable;


In [0]:
%sql
-- Genre dimension
CREATE TABLE netflix_data.data_warehouse.DimGenre (
    GenreID INT PRIMARY KEY,
    GenreName VARCHAR(50)
);

In [0]:
%sql
INSERT INTO netflix_data.data_warehouse.DimGenre(GenreID, GenreName)
VALUES
 (1, 'Comedy'),
 (2, 'Action'),
 (3, 'Adventure'),
 (4, 'Animation'),
 (5, 'Children'),
 (6, 'Fantasy'),
 (7, 'SciFi'),
 (8, 'Drama'),
 (9, 'Thriller'),
 (10, 'Horror'),
 (11, 'War'),
 (12, 'Musical'),
 (13, 'Documentary'),
 (14, 'Crime');


In [0]:
%sql
-- Bridge table (many-to-many relationship)
CREATE TABLE netflix_data.data_warehouse.BridgeMovieGenre (
    MovieID INT,
    GenreID INT,
    PRIMARY KEY (MovieID, GenreID),
    FOREIGN KEY (MovieID) REFERENCES netflix_data.data_warehouse.DimMovie(MovieID),
    FOREIGN KEY (GenreID) REFERENCES netflix_data.data_warehouse.DimGenre(GenreID)
);

### Transform Yes/No genre flags into rows

In [0]:
%sql
-- Comedy
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 1
FROM netflix_data.transformed_data.bigflattable
WHERE Comedy = 'Yes';

-- Action
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 2
FROM netflix_data.transformed_data.bigflattable
WHERE Action = 'Yes';

-- Adventure
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 3
FROM netflix_data.transformed_data.bigflattable
WHERE Adventure = 'Yes';

-- Animation
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 4
FROM netflix_data.transformed_data.bigflattable
WHERE Animation = 'Yes';

-- Children
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 5
FROM netflix_data.transformed_data.bigflattable
WHERE Children = 'Yes';

-- Fantasy
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 6
FROM netflix_data.transformed_data.bigflattable
WHERE Fantasy = 'Yes';

-- SciFi
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 7
FROM netflix_data.transformed_data.bigflattable
WHERE SciFi = 'Yes';

-- Drama
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 8
FROM netflix_data.transformed_data.bigflattable
WHERE Drama = 'Yes';

-- Thriller
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 9
FROM netflix_data.transformed_data.bigflattable
WHERE Thriller = 'Yes';

-- Horror
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 10
FROM netflix_data.transformed_data.bigflattable
WHERE Horror = 'Yes';

-- War
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 11
FROM netflix_data.transformed_data.bigflattable
WHERE War = 'Yes';

-- Musical
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 12
FROM netflix_data.transformed_data.bigflattable
WHERE Musical = 'Yes';

-- Documentary
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 13
FROM netflix_data.transformed_data.bigflattable
WHERE Documentary = 'Yes';

-- Crime
INSERT INTO netflix_data.data_warehouse.BridgeMovieGenre(MovieID, GenreID)
SELECT MovieID, 14
FROM netflix_data.transformed_data.bigflattable
WHERE Crime = 'Yes';


In [0]:
%sql
-- User dimension
CREATE TABLE netflix_data.data_warehouse.DimUser (
    UserID INT PRIMARY KEY
);


In [0]:
%sql
INSERT INTO netflix_data.data_warehouse.DimUser(UserID)
SELECT DISTINCT UserID
FROM netflix_data.transformed_data.bigflattable

In [0]:
%sql
-- Tag Dimension
CREATE TABLE netflix_data.data_warehouse.DimTag (
     TagName VARCHAR(255) PRIMARY KEY,
     TagDateTime DATE
);


In [0]:
%sql
INSERT INTO netflix_data.data_warehouse.DimTag(TagName, TagDateTime)
SELECT DISTINCT MovieTag, MovieTagDateTime
FROM netflix_data.transformed_data.bigflattable

In [0]:
%sql
--fact Table Movie Interaction
CREATE TABLE netflix_data.data_warehouse.FactMovieInteraction (
    InteractionID BIGINT PRIMARY KEY, -- surrogate key
    UserID INT,
    MovieID INT,
    TagName VARCHAR(255),
    Rating DECIMAL(2,1),
    FOREIGN KEY (UserID) REFERENCES netflix_data.data_warehouse.DimUser(UserID),
    FOREIGN KEY (MovieID) REFERENCES netflix_data.data_warehouse.DimMovie(MovieID),
    FOREIGN KEY (TagName) REFERENCES netflix_data.data_warehouse.DimTag(TagName)
);

In [0]:
%sql
INSERT INTO netflix_data.data_warehouse.FactMovieInteraction(InteractionID, UserID, MovieID, TagName, Rating)
SELECT ROW_NUMBER() OVER (ORDER BY UserID, MovieID, MovieTag, AverageRatingOfMovie) AS InteractionID,
       UserID, MovieID, MovieTag, AverageRatingOfMovie
FROM netflix_data.transformed_data.bigflattable